# **Logistic Regression**
# Importing libraries and initializing variables :

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow

np.random.seed(0)

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/classification_train.csv",index_col=0)
labels = df['label'].values

# Analysis of Labelled Data :

classes = pd.DataFrame([np.unique(df.label.values),np.bincount(df.label.values)],index = ['labels','Number of labels'])
print(classes)

# Showing garyscale images using pixel values of 784 = 28 * 28 :

itr = 0
for i in range(30000):
    if(labels[i] == itr):
        if(itr > 9):
            break
        itr+=1
        xi = df.iloc[i,1:].to_numpy()
        xi = xi.reshape([28,28])
        img = np.uint8(xi)
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        cv2_imshow(img)

# Splitting Labelled data into training and testing sets :

X = df.iloc[:25000,1:].to_numpy(dtype='longdouble')           # Labelled Training data
X_test = df.iloc[25000:,1:].to_numpy()                        # Labelled Test data
label_train = df.iloc[:25000,0].values                        # Training Labels 
label_test = df.iloc[25000:,0].values                         # Testing Labels

row , col = X.shape

# One Hot Encoding of labels :

y = np.zeros([10,row]) # row - wise label                     # Training labels
y_test = np.zeros([10,5000])                                  # Testing labels

for i in range(row):
    y[label_train[i],i] = 1
    
for i in range(X_test.shape[0]):
    y_test[label_test[i],i] = 1

print(X_test.shape)                                           # Checking dimensions
print(y_test.shape)

                     0     1     2     3     4     5     6     7     8     9
labels               0     1     2     3     4     5     6     7     8     9
Number of labels  3000  3000  3000  3000  3000  3000  3000  3000  3000  3000


(5000, 784)
(10, 5000)


# Normalizing features:

In [2]:
x = (X) / 255 
x_test = (X_test)/ 255  

# Defining some useful functions :

In [3]:
def softmax(x):
  softmax = np.exp(x)/np.sum(np.exp(x),axis = 0)
  return softmax

def extrem(y_test_prob_dist):                        # Converts predicted probability distribution into a One Hot Encoded array 
  y_res = np.zeros(y_test_prob_dist.shape)           
  itr = 0
  for i in np.argmax(y_test_prob_dist,axis = 0):
    y_res[i,itr] = 1
    itr+=1

  return y_res

def accuracy(x_test,m,y_test):                             # Accuracy function to check model accuracy over labelled test data
  y_pred = m @ x_test.T                            
  count = 0
  row, col = y_test.shape
  y_prob_dist = softmax(y_pred)
  y_res = extrem(y_prob_dist)

  for i in range(col):
    if(np.argmax(y_res[:,i])==np.argmax(y_test[:,i])):     # compares predicted label to test label element wise 
      count+=1

  score = count/col
  return score

# Implementing Softmax Logistic Regression :

In [10]:
def softmax_reg(no_of_iterations,learning_rate,x, y ,x_test,y_test):
    row , col = x.shape

    start = 0 
    end = row 
    n_batch = 10                                                 
    step = row//n_batch                                # mini-batch gradient descent
    
    m = np.random.randn(10,col)                        # randomly initializing weights

    '''Comment out below m to initialize weights with random values.'''

    m = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/logistic_m.csv",index_col = 0) 
    m = m.to_numpy()         
                                                       # using stored value of weights
    y_pred = np.ones((10,row))

    itr = 0
    for i in range(no_of_iterations):
        Jw = 0
        for k in range(start , end , step): 
            x_sec = x[k:k+step,:]
            y_sec = y[:,k:k+step].reshape([10,step])

            y_pred = np.dot( m , x_sec.T )                          # hypothesis 
            y_prob_dist = softmax(y_pred)               # applying softmax to get probabilty distribution for each label on each set of inputs

            loss = -1/row * np.sum( y_sec * np.log(y_prob_dist) + (1-y_sec) * np.log(1-y_prob_dist) )   # cross entropy loss

    # Update Rule :

            Jw += np.dot( (y_sec-y_prob_dist) , x_sec )
            m += learning_rate/row * (Jw/n_batch)
    
            itr += 1
            if (itr+1) % 10 == 0:
                print("After {} iterations :".format(itr+1))
                print("loss = ", loss)

                score = accuracy( x_test , m , y_test )              # Calculating Score against labelled test data
                print("Accuracy = ", score*100 , "%")

                df_m = pd.DataFrame(m)                               # storing weights into a csv file 
                df_m.to_csv("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/logistic_m.csv")
        

softmax_reg(no_of_iterations=10,learning_rate=0.1,x=x,y=y,x_test=x_test,y_test = y_test)

After 10 iterations :
loss =  0.06078458565486272737
Accuracy =  83.06 %
After 20 iterations :
loss =  0.060784638906696369166
Accuracy =  83.06 %
After 30 iterations :
loss =  0.060784693617841329336
Accuracy =  83.06 %
After 40 iterations :
loss =  0.060784749718793844385
Accuracy =  83.06 %
After 50 iterations :
loss =  0.060784807143618515384
Accuracy =  83.06 %
After 60 iterations :
loss =  0.060784865829754862306
Accuracy =  83.06 %
After 70 iterations :
loss =  0.06078492571783512553
Accuracy =  83.06 %
After 80 iterations :
loss =  0.060784986751512548942
Accuracy =  83.06 %
After 90 iterations :
loss =  0.060785048877299479646
Accuracy =  83.06 %
After 100 iterations :
loss =  0.060785112044414744065
Accuracy =  83.06 %


# Predictions on Unlabelled data :

In [ ]:
# Using stored value of Weights

m = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/logistic_m.csv",index_col = 0)
m = m.to_numpy()                                                       

df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/classification_test.csv")

Test_data = df_test.iloc[:,1:].values                          # Unlabelled data
ids = df_test.iloc[:,0]

test_data = (Test_data- np.mean(X,axis =0 ))/255               # Standardization w.r.t Training data

y_test_pred = m @ test_data.T                                  
y_test_prob_dist = softmax(y_test_pred)                        # Predicting labels
y_test_res = extrem(y_test_prob_dist)

print((y_test_res).T, "\n" ,(y_test_res).T.shape)              # Checking dimensions

res_labels = np.argmax(y_test_prob_dist,axis = 0)
res = pd.DataFrame([ids,res_labels]).T                        
res.columns = ["Ids", "Predicted Label"]                       # Storing Result in csv
print(res)                                                     
res.to_csv("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Logistic_Regression_Result.csv") 


[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]] 
 (10000, 10)
        Ids  Predicted Label
0     25672                0
1     59964                1
2     66230                2
3     50801                0
4     83307                1
...     ...              ...
9995  71009                0
9996  67472                6
9997  99226                8
9998  58519                8
9999  80458                3

[10000 rows x 2 columns]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
